## Use of different Aggregators

This notebook shows how to use different Aggregators (FedAvg, FedAdam, FedYogi, FedAdaGrad). 

When you start this tutorial you should either have an account and project in FEDn Studio, or have deployed a FEDn in pseudo-distributed  mode. You should also have created the compute package and the initial model, see README.md for instructions.

    
Note that this notebook is intended to showcase the aggregator API. Fine-tuning of the server-side hyperparameters would be necessary for optimal performance and will need to be done a use-case basis.

In [5]:
from fedn import APIClient
import time
import uuid
import json
import matplotlib.pyplot as plt
import numpy as np
import collections

We make a client connection to the FEDn API service. Here we assume that FEDn is deployed locally in pseudo-distributed mode with default ports. To connect to Studio, generate an API token from the UI, and retrive the controller host URI from the Dashboard.  

#### If using a local development deployment

In [2]:
CONTROLLER_HOST = "127.0.0.1"
CONTROLLER_PORT = 8092
client = APIClient(CONTROLLER_HOST, CONTROLLER_PORT)

#### If using a FEDn Studio project

In [25]:
# Get the controller host for your project from the Dashboard page in Studio
#CONTROLLER_HOST = "fedn.scaleoutsystems.com/aggtestproject-etq-fedn-reducer"
# Generate an API token from Settings->Generate token (the one below is just an example, it will not work)
#TOKEN = "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.bl90eXBlIjoiYWNjZXNzIiwiZXhwIjoxNzE3MDE5MjIwLCJpYXQiOjE3MTQ0MIjQ0OWYyODQwMDQ3NzQxMzll9pZCI6MzcsImNyZWF0b3IiOiJhbmRyZWFzaCIsInJvbGUiOiJhZG1pbiIsInByb2plY3Rfc2x1ZyI6ImFnZ3Rlc3Rwcm9qZWN0LWV0cSJ9.P1RwQElLy3kx3h2o9uE-TUICT4CLlgrrM9YuRasCrBM"
#client = APIClient(CONTROLLER_HOST, token=TOKEN, secure=True,verify=True)

Initialize FEDn with the compute package and seed model. Note that these files needs to be created separately by follwing instructions in the README.

In [ ]:
client.set_active_package('../mnist-pytorch/package.tgz', 'numpyhelper')
client.set_active_model('../mnist-pytorch/seed.npz')
seed_model = client.get_active_model()
print(seed_model)

### FedAvg

In [ ]:
session_id = "experiment_fedavg"

session_config = {
                    "helper": "numpyhelper",
                    "id": session_id,
                    "model_id": seed_model['model'],
                    "rounds": 10
                 }

result_fedavg = client.start_session(**session_config)

### FedAdam

Here we use the FedOpt family of aggregation algorithms. FEDn support adam, yogi and adagrad as server side optimizers. In the session_config below we illustrate how to set hyperparamters (will be valid for this session). The values below are actually the default values and are passed here for illustrative purposes.

**Note that the server side-momentum terms are only retained within one session - each new session you will re-initialize the optimizer to default values.** 

In [ ]:
session_id = "experiment_fedadam"

session_config = {
                    "helper": "numpyhelper",
                    "id": session_id,
                    "aggregator": "fedopt",
                    "aggregator_kwargs": {
                        "serveropt": "adam",
                        "learning_rate": 1e-2,
                        "beta1": 0.9,
                        "beta2": 0.99,
                        "tau": 1e-4
                        },
                    "model_id": seed_model['model'],
                    "rounds": 10
                 }

result_fedadam = client.start_session(**session_config)

### FedYogi

In [ ]:
session_id = "experiment_fedyogi"

session_config = {
                    "helper": "numpyhelper",
                    "id": session_id,
                    "aggregator": "fedopt",
                    "aggregator_kwargs": {
                        "serveropt": "yogi",
                        "learning_rate": 1e-2,
                        },
                    "model_id": seed_model['model'],
                    "rounds": 10
                 }

result_fedyogi = client.start_session(**session_config)
while not client.session_is_finished(session_config['id']):
    time.sleep(2)

### FedAdaGrad

In [12]:
session_id = "experiment_fedadagrad"

session_config = {
                    "helper": "numpyhelper",
                    "id": session_id,
                    "aggregator": "fedopt",
                    "aggregator_kwargs": {
                        "serveropt": "adagrad",
                        "learning_rate": 1e-1,
                        },           
                    "model_id": seed_model['model'],
                    "rounds": 10
                 }

result_fedadagrad = client.start_session(**session_config)

Next, we get the model trail, retrieve all model validations from all clients, extract the training accuracy metric, and compute its mean value accross all clients.

In [17]:
from collections import OrderedDict
def get_validation_acc(session_id):
    validations = client.get_validations(session_id)
    acc = OrderedDict()
    for validation in validations['result']:
       try:   
           acc[validation['model_id']].append(json.loads(validation['data'])['training_accuracy'])
       except:
           acc[validation['model_id']] = [json.loads(validation['data'])['training_accuracy']]

    accuracy_score = []
    for key, value in acc.items():
        accuracy_score.append(np.mean(value))
    accuracy_score.reverse()
    return(accuracy_score)
    
score = get_validation_acc("experiment_fedadagrad")

In [20]:
mean_acc_fedavg = get_validation_acc("experiment_fedavg")
mean_acc_fedadam = get_validation_acc("experiment_fedadam")
mean_acc_yogi = get_validation_acc("experiment_fedyogi")
mean_acc_adagrad = get_validation_acc("experiment_fedadagrad")

In [ ]:
x = range(1,len(mean_acc_fedavg)+1)
plt.plot(x,mean_acc_fedavg, x, mean_acc_fedadam, x, mean_acc_yogi, x, mean_acc_adagrad)
plt.legend(['FedAvg','FedAdam', 'FedYogi', 'FedAdaGrad'])